In [1]:
# Import Libraries
import requests
from pprint import PrettyPrinter
pp = PrettyPrinter()
import pandas as pd
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("TMDB_API_KEY")
omdb_api_key = os.getenv("OMDB_API_KEY")

Step 1 :Get List of all bollywood movies release afer 2010


In [61]:
def get_movie_list()->pd.DataFrame:
    url = f"https://api.themoviedb.org/3/discover/movie?api_key={api_key}&include_adult=false&include_video=false&sort_by=revenue.desc&with_original_language=hi"
    response = requests.get(url).json()
    total_pages = response['total_pages']
    movies_info = pd.DataFrame()
    for page in range(1,total_pages):
        page_url  = f"{url}&page={page}"
        page_response = requests.get(page_url).json()['results']
        movies_info =pd.concat([movies_info,pd.DataFrame(page_response)])

    movies_info = movies_info[(movies_info['release_date']>='2000-01-01') & (movies_info['release_date']<='2024-04-01')]
    columns_to_keep = ['title','genre_ids','id','popularity','release_date']
    movies_info = movies_info[columns_to_keep]
    return movies_info

In [62]:
movies_info = get_movie_list()

In [64]:
movies_info=movies_info.reset_index()

In [65]:
movies_info.shape

(4033, 7)

In [67]:
movies_info.tail()

,level_0,index,title,genre_ids,id,popularity,release_date
4028,4028,14,Salaam Namaste,"[35, 10749, 18]",15084,7.950,2005-09-09
4029,4029,15,Dasvidaniya,[18],14751,2.030,2008-11-07
4030,4030,16,Omkara,"[80, 18]",14705,7.678,2006-07-28
4031,4031,17,Mumbai Salsa,"[18, 35, 10749]",14548,0.667,2007-10-22
4032,4032,18,Golmaal Returns,[35],14546,6.889,2008-10-29


In [69]:
def get_movie_details(movie_id): 
    url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}"
    response = requests.get(url).json()
    return response

In [70]:
useful_columns = ['budget','id','imdb_id','popularity','revenue','title']
movie_details = pd.DataFrame()
for i in range(len(movies_info)):
    response = get_movie_details(movies_info['id'][i])
    for key,value in list(response.items()):
        if key not in useful_columns:
            del response[key]
    movie_details= pd.concat([movie_details,(pd.DataFrame([response]))])


In [ ]:
movie_details.shape

(2995, 6)

In [60]:
movie_details_filtered = movie_details[(movie_details['revenue']!=0) or (movie_details['budget']!=0)]
movie_details_filtered.shape

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [10]:
def get_movie_cast(movie_id):
    url = f"https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={apikey_tmdb}"
    response = requests.get(url).json()
    return response

In [12]:
get_movie_cast(film_info['id'][4])

{'id': 441889,
 'cast': [{'adult': False,
   'gender': 1,
   'id': 1756862,
   'known_for_department': 'Acting',
   'name': 'Zaira Wasim',
   'original_name': 'Zaira Wasim',
   'popularity': 2.503,
   'profile_path': '/lt7S0HddIyHC6HiyuSD75dJppCX.jpg',
   'cast_id': 0,
   'character': 'Insia"Insu"Malik',
   'credit_id': '58a4bb87c3a3686cd4007f56',
   'order': 0},
  {'adult': False,
   'gender': 2,
   'id': 52763,
   'known_for_department': 'Acting',
   'name': 'Aamir Khan',
   'original_name': 'Aamir Khan',
   'popularity': 7.531,
   'profile_path': '/bI3yxh4FzKL2hHYS4nbzcDMWtZs.jpg',
   'cast_id': 1,
   'character': 'Shakti Kumar',
   'credit_id': '58a4bb94c3a3686cbf007d7f',
   'order': 1},
  {'adult': False,
   'gender': 1,
   'id': 1490392,
   'known_for_department': 'Acting',
   'name': 'Meher Vij',
   'original_name': 'Meher Vij',
   'popularity': 7.531,
   'profile_path': '/5tTiy68VEjs4e0Qz4ANSy7LRGo0.jpg',
   'cast_id': 2,
   'character': 'Najma Malik',
   'credit_id': '58a4bb9e

In [56]:
movie_details = pd.DataFrame()
useful_columns = ['imdb_id','Actors','BoxOffice','Director','Genre','Released','Title','Writer']
for i in range(1000):
    print(i)
    response = requests.get(f"http://www.omdbapi.com/?apikey={apiKey}&t={film_info['title'][i]}&plot=full").json()
    for key,value in list(response.items()):
        if key not in useful_columns:
            del response[key]
    
    movie_detail =pd.DataFrame([response])
    movie_details = pd.concat([movie_details,movie_detail])
    if i == film_info.shape[0]/2:
        print(i)

movie_details.shape

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

(1000, 7)

In [18]:
actors_star_power = {
    'Aamir Khan':[15480],
    'Salman Khan':[16045],
    'Shahrukh Khan':[17705],
    'Akshay Kumar':[14730],
    'Ajay Devgn':[13760],
    'Hrithik Roshan':[12680],
    'Ranbir Kapoor':[5260]
}

{'Actors': 'Sidharth Malhotra, Raashi Khanna, Disha Patani',
 'Awards': 'N/A',
 'BoxOffice': 'N/A',
 'Country': 'India',
 'DVD': 'N/A',
 'Director': 'Sagar Ambre, Pushkar Ojha',
 'Genre': 'Action, Drama, Thriller',
 'Language': 'Hindi',
 'Metascore': 'N/A',
 'Plot': 'After terrorists hijack a passenger plane, an off-duty soldier '
         'aboard devises a strategy to defeat the hijackers and ensure the '
         "passengers' survival when the engine fails.",
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMDdlODdmODItZDAxNi00MjExLWFjOGItMGY3MzFjYzU4NDM3XkEyXkFqcGdeQXVyMTUzNTgzNzM0._V1_SX300.jpg',
 'Production': 'N/A',
 'Rated': 'N/A',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.6/10'}],
 'Released': '15 Mar 2024',
 'Response': 'True',
 'Runtime': '130 min',
 'Title': 'Yodha',
 'Type': 'movie',
 'Website': 'N/A',
 'Writer': 'Sagar Ambre',
 'Year': '2024',
 'imdbID': 'tt16139258',
 'imdbRating': '6.6',
 'imdbVotes': '3,005'}
